In [1]:
from collections import Counter
import numpy as np
import tensorflow.contrib.keras as kr
import tensorflow as tf
import time
from datetime import timedelta
import os
from sklearn import metrics

import moxing as mox
mox.file.shift('os', 'mox')

INFO:root:Using MoXing-v1.14.1-ddfd6c9a
INFO:root:Using OBS-Python-SDK-3.1.2


In [2]:
trainDataPath = "s3://corpus-text-classification1/data/train_5500.label.txt"
testDataPath = "s3://corpus-text-classification1/data/TREC_10.label.txt"
vocabPath = "s3://corpus-text-classification1/data/vocab_dp.txt"
savePath = "s3://corpus-text-classification1/saveModel_dp1/saveModel_dp1"

In [3]:
seq_length = 20
num_classes = 10
embedding_dim = 300

learning_rate = 1e-3
dropout_keep_prob = 0.5

hiddenSizes = [128]
epsilon = 5

num_epochs = 10
batch_size = 128
print_per_batch = 50  # 每多少轮输出一次结果

In [4]:
import string
print(string.punctuation)
trantab = str.maketrans({key: None for key in string.punctuation})
"Hello ,. I's gg".translate(trantab)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


'Hello  Is gg'

In [11]:
def readfile(filePath):
    """读取文件内容，返回文本和标签列表"""
    contents, labels = [], []
    with open(filePath, 'r', encoding='utf-8', errors='ignore') as f:
        for line in f:
            try:
                word = line.strip().split()
                label = word[0].split(":")[0]
                content = word[1:]
                punc = []
                for wi in content:
                    if wi in string.punctuation:
                        # content.remove(content[wi])  不能直接移动
                        punc.append(wi)
                for pi in punc:
                    content.remove(pi)  # 移除标点符号
                
                contents.append(content) 
                labels.append(label)
            except Exception as e:
                print(repr(e))
                pass
    return contents, labels


def readCategory():
    """读取分类目录，固定id"""
    categories = ['ABBR', 'DESC', 'ENTY', 'HUM', 'LOC', 'NUM']
    cat_to_id = dict(zip(categories, range(len(categories))))
    return categories, cat_to_id


def read_vocab(vocabPath):
    """读取词汇表"""
    with open(vocabPath, 'r', encoding='utf-8', errors='ignore') as fp:
        words = [_.strip() for _ in fp.readlines()]
    word_to_id = dict(zip(words, range(len(words))))
    return words, word_to_id


def buildVocab(contents_train, contents_test, vocabPath):
    """根据训练集构建词汇表，存储"""
    # extend都可以用来合并两个列表，不同点在于extend是在原列表修改，而 + 是生成新的列表
    contents_all = contents_train + contents_test
    all_data = []
    for content in contents_all:
        all_data.extend(content)

    # 遍历得到每个单词及其出现的次数，组成字典返回
    counter = Counter(all_data)   # Counter({'?': 2, ',': 2, 'NUM:dist': 1, 'How': 1})
    # 统计得到出现频率最高的前    
    count_pairs = counter.most_common()  # [('?', 2), (',', 2), ('NUM:dist', 1), ('How', 1)]
    words, _ = zip(*count_pairs)  # zip(*) 可以看做是解压，即与zip()相反   ('?', ',', 'NUM:dist', 'How')
    # 添加一个 <PAD> 来将所有文本pad为同一长度
    words = list(words)   # list(Counter(all_data).keys())，但要保证顺序，只能这样了 Counter(a).keys()会改变原来的顺序
    open(vocabPath, 'w', encoding='utf-8', errors='ignore').write('\n'.join(words) + '\n')

In [12]:
contents_train, labels_train = readfile(trainDataPath)
contents_test, labels_test = readfile(testDataPath)
contents_val, labels_val = contents_train[-452:], labels_train[-452:]
contents_train, labels_train = contents_train[:-452], labels_train[:-452]

In [13]:
len(contents_val),len(labels_train),len(contents_test)

(452, 5000, 500)

In [14]:
contents_train[:2]

[['How', 'did', 'serfdom', 'develop', 'in', 'and', 'then', 'leave', 'Russia'],
 ['What', 'films', 'featured', 'the', 'character', 'Popeye', 'Doyle']]

In [15]:
category_test = set(labels_test)
category_train = set(labels_train)
print(len(labels_test),len(category_test))
print(len(labels_train),len(category_train))
category_test,category_train

500 6
5000 6


({'ABBR', 'DESC', 'ENTY', 'HUM', 'LOC', 'NUM'},
 {'ABBR', 'DESC', 'ENTY', 'HUM', 'LOC', 'NUM'})

In [16]:
# 如果不存在词汇表，则重建
if not os.path.exists(vocabPath):
    print('======build vocab=======')
    buildVocab(contents_train, contents_test, vocabPath)
categories, cat_to_id = readCategory()  # cat_to_id {'ABBR': 0, 'DESC': 1, 'ENTY': 2, 'HUM': 3, 'LOC': 4, 'NUM': 5}
words, word_to_id = read_vocab(vocabPath)
vocab_size = len(words)
num_classes = len(categories)

======build vocab=======


In [17]:
num_classes = len(categories)

In [18]:
contents_all = contents_train + contents_test
seq_length = 0
for content in contents_all:
    if seq_length < len(content):
        seq_length = len(content)   # seq_length = 37

In [19]:
seq_length

33

In [20]:
cat_to_id

{'ABBR': 0, 'DESC': 1, 'ENTY': 2, 'HUM': 3, 'LOC': 4, 'NUM': 5}

In [21]:
def process_file(contents, labels, word_to_id, cat_to_id, pad_max_length):
    """
    将文件转换为id表示,并且将每个单独的样本长度固定为pad_max_lengtn
    """
    # contents, labels = readfile(filePath)
    data_id, label_id = [], []
    # 将文本内容转换为对应的id形式
    for i in range(len(contents)):
        data_id.append([word_to_id[x] for x in contents[i] if x in word_to_id])
        label_id.append(cat_to_id[labels[i]])
    # 使用keras提供的pad_sequences来将文本pad为固定长度
    x_pad = kr.preprocessing.sequence.pad_sequences(data_id, pad_max_length)
    ''' https://blog.csdn.net/TH_NUM/article/details/80904900
    pad_sequences(sequences, maxlen=None, dtype=’int32’, padding=’pre’, truncating=’pre’, value=0.) 
        sequences：浮点数或整数构成的两层嵌套列表
        maxlen：None或整数，为序列的最大长度。大于此长度的序列将被截短，小于此长度的序列将在后部填0.
        dtype：返回的numpy array的数据类型
        padding：‘pre’或‘post’，确定当需要补0时，在序列的起始还是结尾补
        truncating：‘pre’或‘post’，确定当需要截断序列时，从起始还是结尾截断
        value：浮点数，此值将在填充时代替默认的填充值0
    '''
    y_pad = kr.utils.to_categorical(label_id, num_classes=len(cat_to_id))  # 将标签转换为one-hot表示
    ''' https://blog.csdn.net/nima1994/article/details/82468965
    to_categorical(y, num_classes=None, dtype='float32')
        将整型标签转为onehot。y为int数组，num_classes为标签类别总数，大于max(y)（标签从0开始的）。
        返回：如果num_classes=None，返回len(y) * [max(y)+1]（维度，m*n表示m行n列矩阵，下同），否则为len(y) * num_classes。
    '''
    return x_pad, y_pad


def get_time_dif(start_time):
    """获取已使用时间"""
    end_time = time.time()
    time_dif = end_time - start_time
    return timedelta(seconds=int(round(time_dif)))

In [22]:
print("Loading training and validation and testing data...")
start_time = time.time()
x_train, y_train = process_file(contents_train, labels_train, word_to_id, cat_to_id, seq_length)  # seq_length = 600
x_val, y_val = process_file(contents_val, labels_val, word_to_id, cat_to_id, seq_length)
x_test, y_test = process_file(contents_test, labels_test, word_to_id, cat_to_id, seq_length)
time_dif = get_time_dif(start_time)
print("Loading data Time usage:", time_dif)

Loading training and validation and testing data...
Loading data Time usage: 0:00:00


In [23]:
contents_val[:3]

[['What', 'is', 'a', 'person', "'s", 'socioeconomic', 'position'],
 ['What',
  'do',
  'you',
  'say',
  'to',
  'a',
  'friend',
  'who',
  'ignores',
  'you',
  'for',
  'other',
  'friends'],
 ['How', 'many', 'yards', 'are', 'in', '1', 'mile']]

In [24]:
x_val[:3]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    1,    2,    5,  248,    7, 1737],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           1,   18,   26,  188,    9,    5,  884,   80,   26,   12,  379],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    6,   19, 6459,   11,    4,  210,  527]],
      dtype=int32)

In [25]:
seq_length,num_classes

(33, 6)

In [26]:
embedding_dim = 300

In [27]:
# ======================================================CNN Model Start===============================================
# 输入内容及对应的标签
input_x = tf.placeholder(tf.int32, [None, seq_length], name='input_x')
input_y = tf.placeholder(tf.float32, [None, num_classes], name='input_y')
# dropout的损失率
keep_prob = tf.placeholder(tf.float32, name='keep_prob')

In [28]:
# 词向量映射;实际上此处的词向量并不是用的预训练好的词向量，而是未经任何训练直接生成了一个矩阵，将此矩阵作为词向量矩阵使用，效果也还不错。
# 若使用训练好的词向量，或许训练此次文本分类的模型时会更快，更好。
embedding = tf.get_variable('embedding', [vocab_size, embedding_dim])  # embedding_dim 350
embedding_inputs = tf.nn.embedding_lookup(embedding, input_x)

In [29]:
num_filters = 256
kernel_size = 5
hidden_dim = 128
learning_rate = 1e-3
dropout_keep_prob = 0.5

num_epochs = 20
batch_size = 64
print_per_batch = 20  # 每多少轮输出一次结果

In [30]:
# CNN layer
conv = tf.layers.conv1d(embedding_inputs, num_filters, kernel_size, name='conv')  # num_filters = 256 这是个啥
''' https://blog.csdn.net/khy19940520/article/details/89934335
tf.layers.conv1d：一维卷积一般用于处理文本数据，常用语自然语言处理中，输入一般是文本经过embedding的二维数据。
    inputs： 输入tensor， 维度(batch_size, seq_length, embedding_dim) 是一个三维的tensor；其中，
        batch_size指每次输入的文本数量；
        seq_length指每个文本的词语数或者单字数；
        embedding_dim指每个词语或者每个字的向量长度；
        例如每次训练输入2篇文本，每篇文本有100个词，每个词的向量长度为20，那input维度即为(2, 100, 20)。
    filters：过滤器（卷积核）的数目
    kernel_size：卷积核的大小，卷积核本身应该是二维的，这里只需要指定一维，因为第二个维度即长度与词向量的长度一致，卷积核只能从上往下走，不能从左往右走，即只能按照文本中词的顺序，也是列的顺序。
'''
# global max pooling layer
gmp = tf.reduce_max(conv, reduction_indices=[1], name='gmp')  # https://blog.csdn.net/lllxxq141592654/article/details/85345864

# 全连接层，后面接dropout以及relu激活
fc = tf.layers.dense(gmp, hidden_dim, name='fc1')  # hidden_dim：128
''' https://blog.csdn.net/yangfengling1023/article/details/81774580
dense ：全连接层  inputs：输入该网络层的数据；units：输出的维度大小，改变inputs的最后一维
'''
fc = tf.nn.dropout(fc, keep_prob)
fc = tf.nn.relu(fc)

# 分类器
logits = tf.layers.dense(fc, num_classes, name='fc2')
y_pred_cls = tf.argmax(tf.nn.softmax(logits), 1)  # 预测类别 tf.argmax：返回每一行或每一列的最大值 1为里面（每一行），0为外面（每一列）

# 损失函数，交叉熵
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=input_y)
loss = tf.reduce_mean(cross_entropy)
# 优化器
optim = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

# 准确率
correct_pred = tf.equal(tf.argmax(input_y, 1), y_pred_cls)
acc = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# ======================================================CNN Model End============================================

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [31]:
def batch_iter(x_pad, y_pad, batch_size):
    """生成批次数据"""
    data_len = len(x_pad)
    num_batch = int((data_len - 1) / batch_size) + 1
    # np.arange()生成0到data_len的等差数列，默认等差为1；np.random.permutation()打乱生成的等差序列的顺序
    # 下面三句语句是为了将训练或测试文本的顺序打乱，因为原文本中每个分类的样本全部挨在一起，这样每个batch训练的都是同一个分类，不太好，打乱后每个batch可包含不同分类
    indices = np.random.permutation(np.arange(data_len))
    x_shuffle = x_pad[indices]
    y_shuffle = y_pad[indices]

    # 返回所有batch的数据
    for i in range(num_batch):
        start_id = i * batch_size
        end_id = min((i + 1) * batch_size, data_len)
        yield x_shuffle[start_id:end_id], y_shuffle[start_id:end_id]
        
        
def evaluate(sess, x_pad, y_pad, loss1, acc1):
    """评估在某一数据上的准确率和损失"""
    data_len = len(x_pad)
    batch_eval = batch_iter(x_pad, y_pad, batch_size)  # 128
    total_loss = 0.0
    total_acc = 0.0
    # print(dropout_keep_prob)
    for x_batch1, y_batch1 in batch_eval:
        batch_len = len(x_batch1)
        feed_dict1 = {input_x: x_batch1, input_y: y_batch1, keep_prob: 1.0}
        lossTmp, accTmp = sess.run([loss1, acc1], feed_dict=feed_dict1)
        total_loss += lossTmp * batch_len
        total_acc += accTmp * batch_len

    return total_loss / data_len, total_acc / data_len

In [32]:
# 创建session
session = tf.Session()
saver = tf.train.Saver()
session.run(tf.global_variables_initializer())

In [33]:
# ======================================================Train Start===============================================
# 训练模型的代码，如果要重新训练则打开注释即可。因为后面调用了已训练好的模型，所以此处先注释掉。
print('Training and evaluating...')
start_time = time.time()
total_batch = 0  # 总批次
best_acc_val = 0.0  # 最佳验证集准确率
last_improved = 0  # 记录上一次提升批次
require_improvement = 600  # 如果超过1000轮未提升，提前结束训练
flag = False

for epoch in range(num_epochs):  # 20
    print('Epoch:', epoch + 1)
    batch_train = batch_iter(x_train, y_train, batch_size)
    for x_batch, y_batch in batch_train:
        feed_dict = {input_x: x_batch, input_y: y_batch, keep_prob: dropout_keep_prob}
        session.run(optim, feed_dict=feed_dict)  # 运行优化
        total_batch += 1

        if total_batch % print_per_batch == 0:
            # 每多少轮次输出在训练集和验证集上的性能
            feed_dict[keep_prob] = 1.0
            loss_train, acc_train = session.run([loss, acc], feed_dict=feed_dict)
            loss_val, acc_val = evaluate(session, x_val, y_val, loss, acc)
            if acc_val > best_acc_val:
                # 保存最好结果
                best_acc_val = acc_val
                last_improved = total_batch
                saver.save(sess=session, save_path=savePath)
                improved_str = '*'
            else:
                improved_str = ''

            time_dif = get_time_dif(start_time)
            msg = 'Iter: {0:>6}, Train Loss: {1:>6.2}, Train Acc: {2:>7.2%},' \
                  + ' Val Loss: {3:>6.2}, Val Acc: {4:>7.2%}, Time: {5} {6}'
            print(msg.format(total_batch, loss_train, acc_train, loss_val, acc_val, time_dif, improved_str))

        if total_batch - last_improved > require_improvement:
            # 验证集正确率长期不提升，提前结束训练
            print("No optimization for a long time, auto-stopping...")
            flag = True
            break  # 跳出循环
    if flag:  # 同上
        break
# ======================================================Train End===============================================

Training and evaluating...
Epoch: 1
Iter:     20, Train Loss:    1.5, Train Acc:  40.62%, Val Loss:    1.5, Val Acc:  40.27%, Time: 0:00:04 *
Iter:     40, Train Loss:    0.9, Train Acc:  70.31%, Val Loss:    1.1, Val Acc:  57.96%, Time: 0:00:08 *
Iter:     60, Train Loss:   0.72, Train Acc:  82.81%, Val Loss:    0.8, Val Acc:  74.56%, Time: 0:00:12 *
Epoch: 2
Iter:     80, Train Loss:   0.53, Train Acc:  84.38%, Val Loss:   0.61, Val Acc:  80.53%, Time: 0:00:16 *
Iter:    100, Train Loss:    0.2, Train Acc:  95.31%, Val Loss:   0.56, Val Acc:  82.08%, Time: 0:00:20 *
Iter:    120, Train Loss:   0.25, Train Acc:  90.62%, Val Loss:   0.52, Val Acc:  82.96%, Time: 0:00:24 *
Iter:    140, Train Loss:   0.21, Train Acc:  93.75%, Val Loss:   0.48, Val Acc:  83.41%, Time: 0:00:28 *
Epoch: 3
Iter:    160, Train Loss:  0.017, Train Acc: 100.00%, Val Loss:   0.49, Val Acc:  84.29%, Time: 0:00:31 *
Iter:    180, Train Loss:  0.065, Train Acc:  98.44%, Val Loss:   0.45, Val Acc:  86.06%, Time: 0:

In [34]:
def evaluate_model():
    # 读取保存的模型
    saver.restore(sess=session, save_path=savePath)
    start_time = time.time()
    print('Testing...')
    loss_test, acc_test = evaluate(session, x_test, y_test, loss, acc)
    msg = 'Test Loss: {0:>6.2}, Test Acc: {1:>7.2%}'
    print(msg.format(loss_test, acc_test))

    test_data_len = len(x_test)
    test_num_batch = int((test_data_len - 1) / batch_size) + 1

    y_test_cls = np.argmax(y_test, 1)  # 获得类别
    y_test_pred_cls = np.zeros(shape=len(x_test), dtype=np.int32)  # 保存预测结果  len(x_test) 表示有多少个文本

    for i in range(test_num_batch):  # 逐批次处理
        start_id = i * batch_size
        end_id = min((i + 1) * batch_size, test_data_len)
        feed_dict = {
            input_x: x_test[start_id:end_id],
            keep_prob: 1.0
        }
        y_test_pred_cls[start_id:end_id] = session.run(y_pred_cls, feed_dict=feed_dict)

    # 评估
    print("Precision, Recall and F1-Score...")
    print(metrics.classification_report(y_test_cls, y_test_pred_cls, target_names=categories))
    '''
    sklearn中的classification_report函数用于显示主要分类指标的文本报告．在报告中显示每个类的精确度，召回率，F1值等信息。
        y_true：1维数组，或标签指示器数组/稀疏矩阵，目标值。 
        y_pred：1维数组，或标签指示器数组/稀疏矩阵，分类器返回的估计值。 
        labels：array，shape = [n_labels]，报表中包含的标签索引的可选列表。 
        target_names：字符串列表，与标签匹配的可选显示名称（相同顺序）。 
        原文链接：https://blog.csdn.net/akadiao/article/details/78788864
    '''

    # 混淆矩阵
    print("Confusion Matrix...")
    cm = metrics.confusion_matrix(y_test_cls, y_test_pred_cls)
    '''
    混淆矩阵是机器学习中总结分类模型预测结果的情形分析表，以矩阵形式将数据集中的记录按照真实的类别与分类模型作出的分类判断两个标准进行汇总。
    这个名字来源于它可以非常容易的表明多个类别是否有混淆（也就是一个class被预测成另一个class）
    https://blog.csdn.net/u011734144/article/details/80277225
    '''
    print(cm)

    time_dif = get_time_dif(start_time)
    print("Time usage:", time_dif)

In [37]:
evaluate_model()

INFO:tensorflow:Restoring parameters from s3://corpus-text-classification1/saveModel_dp1/saveModel_dp1


INFO:tensorflow:Restoring parameters from s3://corpus-text-classification1/saveModel_dp1/saveModel_dp1


Testing...
Test Loss:   0.37, Test Acc:  89.40%
Precision, Recall and F1-Score...
              precision    recall  f1-score   support

        ABBR       0.78      0.78      0.78         9
        DESC       0.84      0.99      0.91       138
        ENTY       0.87      0.73      0.80        94
         HUM       0.91      0.91      0.91        65
         LOC       0.90      0.90      0.90        81
         NUM       0.98      0.91      0.94       113

   micro avg       0.89      0.89      0.89       500
   macro avg       0.88      0.87      0.87       500
weighted avg       0.90      0.89      0.89       500

Confusion Matrix...
[[  7   2   0   0   0   0]
 [  1 136   1   0   0   0]
 [  0  15  69   4   4   2]
 [  0   2   3  59   1   0]
 [  1   1   4   2  73   0]
 [  0   5   2   0   3 103]]
Time usage: 0:00:00


In [38]:
def predict(predict_sentences, word_to_id, cat_to_id, pad_max_length):
    """
    将文件转换为id表示,并且将每个单独的样本长度固定为pad_max_lengtn
    """
    
    data_id = []
    # 将文本内容转换为对应的id形式
    for i in range(len(predict_sentences)):
        data_id.append([word_to_id[x] for x in predict_sentences[i].strip().split() if x in word_to_id])
        
    # 使用keras提供的pad_sequences来将文本pad为固定长度
    x_pad = kr.preprocessing.sequence.pad_sequences(data_id, pad_max_length)
    ''' https://blog.csdn.net/TH_NUM/article/details/80904900
    pad_sequences(sequences, maxlen=None, dtype=’int32’, padding=’pre’, truncating=’pre’, value=0.) 
        sequences：浮点数或整数构成的两层嵌套列表
        maxlen：None或整数，为序列的最大长度。大于此长度的序列将被截短，小于此长度的序列将在后部填0.
        dtype：返回的numpy array的数据类型
        padding：‘pre’或‘post’，确定当需要补0时，在序列的起始还是结尾补
        truncating：‘pre’或‘post’，确定当需要截断序列时，从起始还是结尾截断
        value：浮点数，此值将在填充时代替默认的填充值0
    '''
    feed_dict = {
        input_x: x_pad,
        keep_prob: 1.0
    }
    predict_result = session.run(y_pred_cls, feed_dict=feed_dict)
    return predict_result

In [39]:
cat_to_id

{'ABBR': 0, 'DESC': 1, 'ENTY': 2, 'HUM': 3, 'LOC': 4, 'NUM': 5}

In [40]:
'''
LOC:mount Where are the Rocky Mountains ?
DESC:def What are invertebrates ?
NUM:temp What is the temperature at the center of the earth ?
'''
predict(["Where are the Rocky Mountains ?","What are invertebrates ?","What is the temperature at the center of the earth ?"],word_to_id, cat_to_id, seq_length)

array([4, 1, 5])